In [ ]:
pip install pyLDAvis

In [ ]:
pip install nltk

In [ ]:
pip install spacy

In [ ]:
pip install gensim

In [ ]:
pip install contractions

In [ ]:
import pandas as pd
import unicodedata
import re
import contractions
import string
import re 





In [ ]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
#spacy
import spacy
from nltk.corpus import stopwords

In [ ]:
#vis
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
import pandas as pd
file_path_SYMCA ='./Dataset/SYMCA2.parquet'

tweets_df = pd.read_parquet(file_path_SYMCA)

In [ ]:
tweets_df = tweets_df[tweets_df['Handle'] == '@SouthYorksMCA']

In [ ]:
#DataCleaning and Preprocessing
def to_lowercase(text):
    return text.lower()
#converting every row of the column into lower case 
tweets_df.Tweet=tweets_df.Tweet.apply(to_lowercase)

In [ ]:
#Standardizing Accent Characters
def standardize_accented_chars(text):
 return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#testing the function on a single sample for explaination
print(standardize_accented_chars('Sómě words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.'))
#standardizing accented characters for every row
tweets_df.Tweet=tweets_df.Tweet.apply(standardize_accented_chars)

print(tweets_df.Tweet)

In [ ]:
#To remove URL from the tweets
def get_number_of_urls(documents):
    print("{:.2f}% of documents contain urls".format(sum
(documents.apply(lambda x:x.find('http'))>0)/len
(documents)*100))
# Passing the 'Tweets' column of the dataframe as the argument
print(get_number_of_urls(tweets_df.Tweet)) 


In [ ]:
#Remove the urls from text using the below-mentioned function
def remove_url(text):
 return re.sub(r'https?:\S*', '', text)
#testing the function on a single sample for explaination
print(remove_url('using https://www.google.com/ as an example'))
#removing urls from every row
tweets_df.Tweet=tweets_df.Tweet.apply(remove_url)
print(tweets_df.Tweet)

In [ ]:
#Expanding Contractions : convert each contraction into its expanded form 
def expand_contractions(text):
    expanded_words = [] 
    for word in text.split():
       expanded_words.append(contractions.fix(word)) 
    return ' '.join(expanded_words)
#testing the function on a single sample for explaination
print(expand_contractions("Don't is same as do not"))
#expanding contractions for every row
tweets_df.Tweet=tweets_df.Tweet.apply(expand_contractions)
print(tweets_df.Tweet)

In [ ]:
 # Define the function to remove mentions and hashtags
def remove_mentions_and_tags(text):
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

# Testing the function on a single sample
print(remove_mentions_and_tags('Some random @abc and #def'))

# Removing mentions and tags from every row in the DataFrame
tweets_df['Tweet'] = tweets_df['Tweet'].apply(remove_mentions_and_tags)
print(tweets_df['Tweet'])

In [ ]:


def keep_only_alphabet(text):
    return re.sub(r'[^a-zA-Z]+', ' ', text)

# Testing the function on a single sample
print(keep_only_alphabet(' Just a bit more $$processing required.Just a bit!!!'))

# For all the rows in the DataFrame
tweets_df['Tweet'] = tweets_df['Tweet'].apply(keep_only_alphabet)
print(tweets_df['Tweet'])

In [ ]:
#Removing Stopwords(Default+Custom) and Removing Short Words
def remove_stopwords(text,nlp,custom_stop_words=None,
remove_small_tokens=True,min_len=2):
    # if custom stop words are provided, then add them to default stop words list
    if custom_stop_words:
        nlp.Defaults.stop_words |= custom_stop_words
    
    filtered_sentence =[] 
    doc=nlp(text)
    for token in doc:
        
        if token.is_stop == False: 
            
            # if small tokens have to be removed, then select only those which are longer than the min_len 
            if remove_small_tokens:
                if len(token.text)>min_len:
                    filtered_sentence.append(token.text)
            else:
                filtered_sentence.append(token.text)
    # if after the stop word removal, words are still left in the sentence, then return the sentence as a string else return null 
    return " ".join(filtered_sentence) if len(filtered_sentence)>0 else None

#creating a spaCy object. 
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
#removing stop-words and short words from every row
tweets_df.Tweet=tweets_df.Tweet.apply(lambda x:remove_stopwords(x,nlp,{"SYMCA"}))

In [ ]:
tweets_df['Tweet'].fillna("", inplace=True)  # Fill missing values with empty strings

#LEMETIZATION

def lemmatize(text, nlp):
    doc = nlp(text)
    lemmatized_text = []
    for token in doc:
        lemmatized_text.append(token.lemma_)
    return " ".join(lemmatized_text)

# Testing the function on a single sample
print(lemmatize('Reading NLP blog is fun.', nlp))

# Performing lemmatization on every row
tweets_df['Tweet'] = tweets_df['Tweet'].apply(lambda x: lemmatize(x, nlp))
print(tweets_df['Tweet'])


In [ ]:
#Generating Document Matrix and Dictionary:
def generate_tokens(tweet):
    words=[]
    for word in tweet.split(' '):
    # using the if condition because we introduced extra spaces during text cleaning
        if word!='':
            words.append(word)
        return words
#storing the generated tokens in a new column named 'words'
tweets_df['tokens']=tweets_df.Tweet.apply(generate_tokens)
print(tweets_df['tokens'])

In [ ]:

def create_dictionary(words):
    return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
id2word=create_dictionary(tweets_df.tokens)
print(id2word)

In [ ]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
            corpus.append(id2word.doc2bow(text))
    return corpus

#print(tweets_df.tokens[1])
print(id2word)
corpus=create_document_matrix(tweets_df.tokens,id2word)
print(tweets_df.tokens)
print(corpus)

In [ ]:
#Implementing LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
 id2word=id2word,
 num_topics=10,
 random_state=100,
 )

In [ ]:
#Genrating LDA topics
def get_lda_topics(model, num_topics, top_n_words):
     word_dict = {}
     for i in range(num_topics):
         word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
 
     return pd.DataFrame(word_dict)
get_lda_topics(lda_model,5,10)

In [ ]:
import pandas as pd

def get_lda_topics(model, num_topics, top_n_words):
    topic_data = []
    for i in range(num_topics):
        topic_words = [(word, weight) for word, weight in model.show_topic(i, topn=top_n_words)]
        topic_data.append(topic_words)
 
    return pd.DataFrame(topic_data, columns=[f"Keyword #{i+1}" for i in range(top_n_words)])

# Assuming you have already trained your LDA model and named it lda_model
# Adjust the number of topics and top n words accordingly
num_topics = 10
top_n_words = 10

# Get the LDA topics with keywords and weights using your modified function
lda_topics_df = get_lda_topics(lda_model, num_topics, top_n_words)

# Display the DataFrame
print(lda_topics_df)


In [ ]:
import pyLDAvis.gensim
import gensim
import gensim.corpora as corpora

# Assuming you have already defined lda_model, corpus, and id2word

# Create the LDA visualization
lda_vis_data = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

# Display the visualization
pyLDAvis.display(lda_vis_data)
